# Bail or No Bail


## Idea:
Los Angeles County plans bring back a Zero-Bail policy for those arrested on misdemeanors and non-violent offenders. Many in the county had concerns that when this policy was originally enacted it would have caused a spike in crime with those individuals let go becoming re-offenders. This no bail policy was first implemented during the COVID-19 pandemic starting on April 13, 2020 due to concerns of spreading COVID-19 in overcrowded jails. This was always planned to be a temporary measure due to the nature of the pandemic and the zero-bail policy ended on July 1, 2020. On October 1, 2023 Los Angeles County Police Departments will go back to the Zero-Bail Policy with no designated end-date.

### Project
I want to observe and analyze the rates of crimes during the enactment of this policy and prior to the policy. 

### Data Sources:

https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8

https://data.lacity.org/Public-Safety/Crime-Data-from-2010-to-2019/63jg-8b9z

https://fred.stlouisfed.org/series/PPAACA06037A156NCEN

https://fred.stlouisfed.org/series/PE5T17CA06037A647NCEN

https://fred.stlouisfed.org/series/CALOSA7URN

https://fred.stlouisfed.org/series/CALOSA7LFN

https://fred.stlouisfed.org/series/LAUCN060370000000005

### References:
    
https://engineering.stanford.edu/magazine/article/can-ai-help-judges-make-bail-system-fairer-and-safer

https://www.wired.com/story/algorithms-supposed-fix-bail-system-they-havent/

https://www.latimes.com/archives/la-xpm-2005-aug-30-me-crime30-story.html

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
# Function to create a dataframe & dictionary to give 
# us in depth of info of the dataset's 
# datatypes consistency per column
def check_type(df):
    col_type_dict = {}
    # iterate through columns
    for col in list(df.columns.values):
        # create list of types of each cell for column
        type_list = list(map(str,list(df[col].map(type))))
        # find most common type in column
        common = max(set(type_list), key=type_list.count)
        # find total number of that type
        com_num = sum(t == common for t in type_list)
        # list all types found in this column
        all_types = list(set(type_list))
        #compile info into dict
        col_type_dict[col] = (common,com_num,(com_num/len(type_list)),all_types)
    
    # create a pandas dataframe, it's easier to read than a dictionary in my opinion
    temp_df = pd.DataFrame.from_dict(col_type_dict, orient='index',\
                                  columns=['most_common_type','most_common_freq',\
                                           'most_common_%','all_types_found'])
    return temp_df

In [5]:
current_df = pd.read_csv('Crime_Data_from_2020_to_present.csv')
current_df.head(5)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,04/14/2020 12:00:00 AM,02/13/2020 12:00:00 AM,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468


In [8]:
check_type(current_df)

,most_common_type,most_common_freq,most_common_%,all_types_found
DR_NO,<class 'int'>,802956,1.000000,[<class 'int'>]
Date Rptd,<class 'str'>,802956,1.000000,[<class 'str'>]
DATE OCC,<class 'str'>,802956,1.000000,[<class 'str'>]
TIME OCC,<class 'int'>,802956,1.000000,[<class 'int'>]
AREA,<class 'int'>,802956,1.000000,[<class 'int'>]
AREA NAME,<class 'str'>,802956,1.000000,[<class 'str'>]
Rpt Dist No,<class 'int'>,802956,1.000000,[<class 'int'>]
Part 1-2,<class 'int'>,802956,1.000000,[<class 'int'>]
Crm Cd,<class 'int'>,802956,1.000000,[<class 'int'>]
Crm Cd Desc,<class 'str'>,802956,1.000000,[<class 'str'>]


In [10]:
# view number of NaN values in dataset
current_df.isna().sum()

DR_NO                  0
Date Rptd              0
DATE OCC               0
TIME OCC               0
AREA                   0
AREA NAME              0
Rpt Dist No            0
Part 1-2               0
Crm Cd                 0
Crm Cd Desc            0
Mocodes           110652
Vict Age               0
Vict Sex          105273
Vict Descent      105281
Premis Cd              9
Premis Desc          475
Weapon Used Cd    523431
Weapon Desc       523431
Status                 0
Status Desc            0
Crm Cd 1              10
Crm Cd 2          743809
Crm Cd 3          800986
Crm Cd 4          802899
LOCATION               0
Cross Street      674434
LAT                    0
LON                    0
dtype: int64

In [12]:
prev_df = pd.read_csv('Crime_Data_from_2010_to_2019.csv')
prev_df.head(5)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,...,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524
3,90631215,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,...,IC,Invest Cont,900.0,998.0,NaN,NaN,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295
4,100100501,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",...,IC,Invest Cont,122.0,NaN,NaN,NaN,8TH ST,SAN PEDRO ST,34.0387,-118.2488


In [13]:
check_type(prev_df)

,most_common_type,most_common_freq,most_common_%,all_types_found
DR_NO,<class 'int'>,2136018,1.000000,[<class 'int'>]
Date Rptd,<class 'str'>,2136018,1.000000,[<class 'str'>]
DATE OCC,<class 'str'>,2136018,1.000000,[<class 'str'>]
TIME OCC,<class 'int'>,2136018,1.000000,[<class 'int'>]
AREA,<class 'int'>,2136018,1.000000,[<class 'int'>]
AREA NAME,<class 'str'>,2136018,1.000000,[<class 'str'>]
Rpt Dist No,<class 'int'>,2136018,1.000000,[<class 'int'>]
Part 1-2,<class 'int'>,2136018,1.000000,[<class 'int'>]
Crm Cd,<class 'int'>,2136018,1.000000,[<class 'int'>]
Crm Cd Desc,<class 'str'>,2136018,1.000000,[<class 'str'>]
